In [ ]:
import json
import numpy as np
import torch
from torch import cuda
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM

def stream_jsonl(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    np.random.shuffle(lines)
    
    for line in lines:
        yield json.loads(line)
            
file_paths = ['./data/helpful-base/test.jsonl', './data/harmless-base/test.jsonl']

model_id = 'princeton-nlp/Sheared-LLaMA-2.7B'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
device

In [ ]:
# begin initializing HF items, need auth token for these
model_config = AutoConfig.from_pretrained(
    model_id,
    # use_auth_token=hf_auth
)

with init_empty_weights():
    model = AutoModelForCausalLM.from_config(model_config)
# print(model)

device_map = infer_auto_device_map(model, no_split_module_classes=["LlamaDecoderLayer"])
device_map

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    # use_auth_token=hf_auth
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
    # use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

In [ ]:
data_path = file_paths[np.random.choice([0,1])]
prompt = next(iter(stream_jsonl(data_path)))
prompt = prompt['chosen'].split('\n')[2]
print(prompt)

In [ ]:
# Tokenize input text
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text
with torch.no_grad():
    output = model.generate(input_ids,
                            max_length=100,
                            num_return_sequences=1,
                            temperature=0.7,
                            pad_token_id=tokenizer.eos_token_id)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)